# About this example

This example shows how you can deploy DistilBERT model to analyze confidential text for sentiment analysis. The model will be left untrained for demonstration purposes. We could finetune it on positive/negative samples before deploying it.

By using BlindAI, people can send data for the AI to analyze sensitive text without having to fear privacy leaks.

DistilBERT is a state of the art Transformers model for NLP. You can learn more about it on this [Hugging Face page](https://huggingface.co/docs/transformers/model_doc/distilbert).

More information on this use case can be found on our blog post [Deploy Transformers models with confidentiality](https://blog.mithrilsecurity.io/transformers-with-confidentiality/).

# Installing dependencies

Install the dependencies this example needs.

In [1]:
!pip install transformers[onnx] torch

Install the latest version of BlindAI.

In [3]:
!pip install blindai

# Preparing the model

In this first step we will export a standard Hugging Face DistilBert model to an ONNX file, as BlindAI accepts only ONNX files.

In [1]:
from transformers import DistilBertForSequenceClassification

# Load the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

Then, we need to export this Pytorch model to ONNX. To do the export, we need an example for dynamic tracing to see how an input is processed by the graph. This may take a little while.

We have fixed a max length of 8 for simplicity. Having a fixed max size makes it easier for deployment. If the input is shorter, we can simply pad it to the max size.

In [2]:
from transformers import DistilBertTokenizer
import torch

# Create dummy input for export
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "I love AI and privacy!"
inputs = tokenizer(sentence, padding = "max_length", max_length = 8)["input_ids"]

# Export the model
torch.onnx.export(
	model, inputs, "./distilbert-base-uncased.onnx",
	export_params=True, opset_version=11,
	input_names = ['input'], output_names = ['output'],
	dynamic_axes={'input' : {0 : 'batch_size'},
	'output' : {0 : 'batch_size'}})

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

# Deployment on BlindAI

Please make sure the **server is running**. To launch the server, refer to the [Launching the server](https://docs.mithrilsecurity.io/getting-started/quick-start/run-the-blindai-server) documentation page. 

If you have followed the steps and have the Docker image ready, this mean you simply have to run `docker run -it -p 50051:50051 -p 50052:50052 mithrilsecuritysas/blindai-server-sim:latest`

So the first thing we need to do is to connect securely to the BlindAI server instance. Here we will use simulation mode for ease of use. This means that we do not leverage the hardware security propertiers of secure enclaves, but we do not need to run the Docker image with a specific hardware.

If you wish to run this example in hardware mode, you need to prepare the `host_server.pem` and `policy.toml` files. Learn more on the [Deploy on Hardware](https://docs.mithrilsecurity.io/getting-started/deploy-on-hardware) documentation page. 

In [3]:
from blindai.client import BlindAiClient, ModelDatumType

# Launch client
client = BlindAiClient()

# Simulation mode
client.connect_server(addr="localhost", simulation=True)

# Hardware mode
# client.connect_server(addr="localhost", policy="./policy.toml", certificate="./host_server.pem")

Then, upload the model inside the BlindAI server. This simply means uploading the ONNX file created before.

When uploading the model, we have to precise the shape of the input and the data type. In this case, because we use Transformers model with tokens, we simply need to send the indices of the tokens, i.e. integers.

In [4]:
client.upload_model(model="./distilbert-base-uncased.onnx", shape=inputs.shape, dtype=ModelDatumType.I64)

# Sending data for confidential prediction

Now it's time to check it's working live!

We will just prepare some input for the model inside the secure enclave of BlindAI to process it.

In [5]:
from transformers import DistilBertTokenizer

# Prepare the inputs
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "I love AI and privacy!"
inputs = tokenizer(sentence, padding = "max_length", max_length = 8)["input_ids"]

Now we can get a prediction.

In [6]:
response = client.run_model(inputs)
response.output

[-0.027642814442515373, -0.0573299303650856]

Here we can compare the results against the original prediction.

In [7]:
model(torch.tensor(inputs).unsqueeze(0)).logits.detach()

tensor([[-0.0276, -0.0573]])

Et voila! We have been able to apply a start of the art model of NLP, without ever having to show the data in clear to the people operating the service!

If you have liked this example, do not hesitate to drop a star on our [GitHub](https://github.com/mithril-security/blindai) and chat with us on our [Discord](https://discord.gg/TxEHagpWd4)!